# **Analytical solution for 2D continuous solute transport injection in porous media (with decay)**

Here, we'll see the evolution in time of the concentracion of five different contaminants, each with a different $\lambda$ value.

- 0.0001 day⁻¹ → very slow decay (e.g., nitrate in oxic groundwater, half-life ≈ 19 years)
- 0.001 day⁻¹ → moderate-slow decay (e.g., chlorinated solvents, half-life ≈ 1.9 years)
- 0.01 day⁻¹ → moderate decay (e.g., BTEX compounds biodegradation, half-life ≈ 70 days)
- 0.1 day⁻¹ → relatively fast decay (e.g., pathogens, half-life ≈ 1 week)
- 1.0 day⁻¹ → very fast decay (e.g., labile organic matter, half-life ≈ 0.7 days)


And the equation that defines 2D continuous solute transport injection, with decay, in porous media is the following,

$C(x, y, t) = \frac{M/T}{4\pi \phi \left( D'_{L}D'_{T}\right)^{1/2}}\operatorname{exp}\!\left( \frac{x}{B}\right) W\!\left(u, \frac{r}{B}\right)$

where,

$B = \frac{2D'_{L}}{v'}$

$d = 1 + \frac{2B\lambda}{v'}$

$u = \frac{r^{2}}{4dD'_{L}t}$

$r = \sqrt{\left( x^{2} + y^{2} \frac{D'_{L}}{D_{T}} \right)}$

and, when $r/B > 1$, $W\left(u, \frac{r}{B}\right)$ can be approximate as,

$W\left( u, \frac{r}{B} \right) \approx \sqrt{\frac{\pi B}{2r}\operatorname{exp}\!\left( - \frac{r}{B} \right)} \operatorname{erfc}\!\left( \frac{2u - r/B}{2\sqrt{u}} \right)$ 

**References:**
- Bear, J., & Cheng, A. H. D. (2010). Modeling groundwater flow and contaminant transport (Vol. 23, p. 834). Dordrecht: Springer.
- Bear, J. (2012). Hydraulics of groundwater. Courier Corporation.

<br>

## **1. Loading the needed libraries**

In [1]:
#-- Check and install required packages if not already installed 
import sys
import subprocess

def if_require(package):
    try:
        __import__(package)
    except ImportError:
        print(f"{package} not found. Installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        
#-- Install packages only if they aren't already installed 
if_require("ipywidgets")
if_require("openpyxl")
if_require("xlrd")
if_require("numpy")
if_require("scipy")
if_require("os")

#-- Import the rest of libraries
import matplotlib.pyplot as plt
import ipywidgets as widgets
import numpy as np
import scipy as sp
import os

from IPython.display import display, clear_output, HTML
from ipywidgets import Layout, Output
from tkinter import Tk, filedialog

import warnings
warnings.filterwarnings("ignore")

#-- LaTeX configuration, if installed
# plt.rcParams.update({
#     "text.usetex": True,
#     "font.family": "Computer Modern Roman"
# })

#-- Widgets parameters configuration
param_widgets = { 
                 'readout_format': ".2f", 
                 'style': {'description_width': '200px'}, 
                 'layout': widgets.Layout(width='500px')
                 }  

## **2. Coding the equations and plotting the results**

In [ ]:
def effective_velocity(v: float, R:float) -> float:
    """Calculates effective velocity.

    Parameters
    ----------
    v : float
        velocity [L/T]
    R : float
        Retardation factor [-]

    Returns
    -------
    float
        Effective velocity [L/T]
    """
    
    v_prime = v / R
    return v_prime

def effective_dispersion(v: float, R: float, alpha: float, alpha_tran: float | None = None) -> float | tuple[float, float]:
    """Calculates effective dispersion coefficients considering retardation.
    
    When dispersion coefficient is None, then the function would give a value for both the longitudinal
    and transversal dispersion. When these two latter parameters get None, then dispersion coefficient 
    would be returned. 

    Parameters
    ----------
    v : float
        Groundwater velocity [L/T]
    R : float
        Retardation [-]
    alpha : float
        Dispersivity [L]
    alpha_tran : float | None, optional
        Transversal dispersivity, by default None (when 1D)

    Returns
    -------
    float | tuple[float, float]
        - If only `alpha` and `R` are provided:
            Effective longitudinal dispersion D' [L²/T].
        - If `alpha` and `alpha_tran` are provided:
            Tuple (Effective longitudinal dispersion, Effective transversal dispersion) where each 
            coefficient has been divided by R.
        
    Notes
    -----
    The retardation factor R must be ≥ 1. For R = 1, no retardation occurs and
    effective dispersion equals the original dispersion coefficients.
    """
    
    D_prime = (alpha*v) / R
    if alpha_tran is None:
        return D_prime
    else:
        D_tran_prime = (alpha_tran*v) / R
        return D_prime, D_tran_prime


#-- B parameter 
def B_parameter(D_long_prime: float, v_prime: int) -> float:
    """Calculates the B parameter. 
    
    From Bear & Cheng, 2010 (Eq. 7.4.57)

    Parameters
    ----------
    D_long_prime : float
        Effective longitudinal dispersion coefficient [L²/T]
    v_prime : int
        Effective groundwater velocity [L/T]

    Returns
    -------
    float
        B parameter [L]
    """
    B = (2*D_long_prime)/v_prime
    return B

#-- d parameter
def d_parameter(B: float, v_prime: float, decay: float) -> float:
    """Calculates the d parameter. 
    
    From Bear & Cheng, 2010 (Eq. 7.4.57)

    Parameters
    ----------
    B : float
        B parameter [L]
    v_prime : float
        Effective groundwater velocity [L/T]
    decay : float
        Coefficient of radioactive decay [1/T]

    Returns
    -------
    float
        d parameter [-]
    """
    
    d = 1 + ((2*B*decay)/v_prime)
    return d

#-- radius
def radius(x: float, y: float, D_long_prime: float, D_tran_prime: float, d: float) -> float:
    """Calculates the radius.
    
    From Bear & Cheng, 2010 (Eq. 7.4.57)

    Parameters
    ----------
    x : float
        Position in x axis [L]
    y : float
        Position in y axis [L]
    D_long : float
        Longitudinal dispersion coefficient [L²/T]
    D_tran : float
        Transversal dispersion coefficient [L²/T]

    Returns
    -------
    float
        Radius [L]
    """

    r = np.sqrt( ((x**2) + (y**2*(D_long_prime/D_tran_prime))) * d ) 
    return r

#-- Dimensionless time (u)
def dimensionless_time(r:float, d:float, D_long_prime:float, t:float) -> list:
    """Calculates the dimensionless time (u) and returns a list of its values.
    
    From Bear & Cheng, 2010 (Eq. 7.4.57)

    Parameters
    ----------
    r : float
        Radius [L]
    d : float
        d parameter [-]
    D_long : float
        Longitudinal dispersion coefficient [L²/T]
    t : float
        time

    Returns
    -------
    list
        Dimensionless time (u) [-]
    """

    u = (r**2) / (4*d*D_long_prime*t)
    return u


#-- Solving Well function (W) by means of Exponential Integral Approximation
def well_function(B: float, r: float, u:float):
    """Approximation of the well function (W) value and returns a list of its values.

    Parameters
    ----------
    B : float
        B parameter [L]
    r : float
        radius [L]
    u : float
        dimensionless time [-]

    Returns
    -------
    float
        Well function
    """
    
    W = np.sqrt((np.pi * B)/(2*r)) * np.exp(-(r/B)) * sp.special.erfc(((2*u)-(r/B))/(2*np.sqrt(u)))
    
    return W

#-- Continuous Injection: Solving solute transport in an infinite 2D domain with point source
def twoD_continuous_inj(M: float, T: int, x: float, y: float, t: float, v: float, alpha: float, 
                        alpha_tran: float, R:float, phi: float, decay: float) -> np.ndarray:
    """Calculates the concentration in a two-dimensional continuous injection framework.

    Parameters
    ----------
    M : float
        Total mass flux [M/T]
    T : int
        Injection time
    x : float
        Position in x axis [L]
    y : float
        Position in y axis [L]
    t : float
        time
    v : float
        Groundwater velocity [L/T]
    alpha : float
        Longitudinal dispersivity [L]
    alpha_tran : float
        Transversal dispersivity [L]
    R : float
        Retardation coefficient [-]
    phi : float
        Porosity
    decay : float
        Coefficient of radioactive decay [1/T]

    Returns
    -------
    np.ndarray
        Concentration [M/V]
    """
    
    v_prime = effective_velocity(v, R)
    D_long_prime, D_tran_prime = effective_dispersion(v, R, alpha, alpha_tran)
    B = B_parameter(D_long_prime, v_prime)
    d = d_parameter(B, v_prime, decay)
    r = radius(x, y, D_long_prime, D_tran_prime, d)
    u = dimensionless_time(r, d, D_long_prime, t) 
    W = well_function(B, r, u)
    
    # Bear & Cheng (2010), Eq. 7.4.56
    C = ((M * T) / (4*np.pi*phi*np.sqrt(D_long_prime*D_tran_prime))) * np.exp(x/B) * W
      
    return C

def find_parameters_v1(x, y, M, alpha, alpha_tran, R):   
        decay_values = [0.0001, 0.001, 0.01, 0.1, 1]
        names_list = ["0.0001 [1/d]", "0.001 [1/d]", "0.01 [1/d]", "0.1 [1/d]", "1 [1/d]"]
        
        C = [twoD_continuous_inj(M, 365*2, x, y, np.arange(0, 365*2), 1, alpha, alpha_tran, R, 0.3, decay) for decay in decay_values]
 
        #-- Plotting the results
        fig, ax1 = plt.subplots(figsize=(10, 3.5))
                        
        for contaminant in range(len(C)):
                ax1.plot(np.arange(0, 365*2), C[contaminant], label=names_list[contaminant])
        ax1.set_xlim(0, 365*2)
        # ax1.set_ylim(0, 1500)
        ax1.set_xlabel(r'time [days]')
        ax1.set_ylabel('Concentration')
        ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        box = ax1.get_position()
        ax1.set_position([box.x0, box.y0, box.width * 1.1, box.height])
        

sliders = [
        widgets.FloatSlider(value=75, min=50, max=100, step=5, description=r'Mass injected [g]', **param_widgets),
        widgets.FloatSlider(value=250, min=50, max=2000, step=10, description=r'Distance x to obs. point [m]', **param_widgets),
        widgets.FloatSlider(value=0, min=0, max=500, step=10, description=r'Distance y to obs. point [m]', **param_widgets),
        widgets.FloatSlider(value=1, min=0.2, max=10, step=0.1, description=r'Longitudinal dispersivity [m]', **param_widgets),
        widgets.FloatSlider(value=0.1, min=0.02, max=1, step=0.001, description=r'Transversal dispersivity [m]', **param_widgets),
        widgets.FloatSlider(value=1, min=1, max=5, step=0.1, description=r'Retardation [-]', **param_widgets)
        ]

ui = widgets.HBox([widgets.VBox(sliders)])

out = widgets.interactive_output(find_parameters_v1, {
        'M': sliders[0],
        'x': sliders[1],
        'y': sliders[2],
        'alpha': sliders[3],
        'alpha_tran': sliders[4],
        'R': sliders[5]
        })

display(ui, out)

Output()

In [ ]:
def find_parameters_v2(M, t, y, alpha, alpha_tran, R):   
            
        decay_values = [0.0001, 0.001, 0.01, 0.1, 1]
        names_list = ["0.0001 [1/d]", "0.001 [1/d]", "0.01 [1/d]", "0.1 [1/d]", "1 [1/d]"]
        
        C = [twoD_continuous_inj(M, 365*2, np.arange(0, 2000), y, t, 1, alpha, alpha_tran, R, 0.3, decay) for decay in decay_values]
 
        #-- Plotting the results
        fig, (ax1) = plt.subplots(figsize=(10, 5))
        # fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 10))
                        
        #-- Plot 1D
        for contaminant in range(len(C)):
                ax1.plot(np.arange(0, 2000), C[contaminant], label=names_list[contaminant])
        ax1.set_xlim(0, 1000)
        # ax1.set_ylim(0.001, 100000)
        ax1.set_xlabel(r'Distance x [m]')
        ax1.set_ylabel('Concentration')
        ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        box = ax1.get_position()
        ax1.set_position([box.x0, box.y0, box.width * 1.1, box.height])
                  
sliders = [
        widgets.FloatSlider(value=100, min=1, max=1000, step=1, description=r'time [d]', **param_widgets),
        widgets.FloatSlider(value=75, min=50, max=100, step=5, description=r'Mass injected [g]', **param_widgets),
        widgets.FloatSlider(value=0, min=0, max=500, step=10, description=r'Distance y to obs. point [m]', **param_widgets),
        widgets.FloatSlider(value=1, min=0.2, max=10, step=0.1, description=r'Longitudinal dispersivity [m]', **param_widgets),
        widgets.FloatSlider(value=0.1, min=0.02, max=1, step=0.001, description=r'Transversal dispersivity [m]', **param_widgets),
        widgets.FloatSlider(value=1, min=1, max=5, step=0.1, description=r'Retardation [-]', **param_widgets)
        ]

ui = widgets.HBox([widgets.VBox(sliders)])

out = widgets.interactive_output(find_parameters_v2, {
        't': sliders[0],
        'M': sliders[1],
        'y': sliders[2],      
        'alpha': sliders[3],
        'alpha_tran': sliders[4],
        'R': sliders[5]
        })

display(ui, out)

Output()